In [49]:
import pandas as pd
from sqlalchemy import create_engine

# Set up the database connection
server_name = '----'
database_name = '-----'
username = ''
password = ''

connection_string = f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

# Define your SQL query
sql_query = """
    SELECT
        order_id,
        category_name,
        brand_name,
        product_name,
        quantity,
        i.list_price
    FROM
        sales.order_items i
        JOIN production.products p ON p.product_id = i.product_id
        JOIN production.categories c ON c.category_id = p.category_id
        JOIN production.brands b ON b.brand_id = p.brand_id
"""

# Fetch data from the database and load it into a DataFrame
df = pd.read_sql_query(sql_query, engine)

# Print the first few rows of the DataFrame
df


,order_id,category_name,brand_name,product_name,quantity,list_price
0,1,Cruisers Bicycles,Electra,Electra Townie Original 7D EQ - Women's - 2016,1,599.99
1,1,Mountain Bikes,Trek,Trek Remedy 29 Carbon Frameset - 2016,2,1799.99
2,1,Cyclocross Bicycles,Surly,Surly Straggler - 2016,2,1549.00
3,1,Cruisers Bicycles,Electra,Electra Townie Original 7D EQ - 2016,2,599.99
4,1,Mountain Bikes,Trek,Trek Fuel EX 8 29 - 2016,1,2899.99
...,...,...,...,...,...,...
4640,1614,Road Bikes,Trek,Trek Emonda ALR 6 - 2018,2,2299.99
4641,1614,Cruisers Bicycles,Electra,Electra Cruiser 1 - 2016/2017/2018,2,269.99
4642,1615,Electric Bikes,Trek,Trek Verve+ Lowstep - 2018,2,2299.99
4643,1615,Cruisers Bicycles,Electra,Electra Tiger Shark 3i - 2018,1,899.99


In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Set up the database connection
server_name = '-----'
database_name = '-----'
username = ''
password = ''

connection_string = f'mssql+pyodbc://{username}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(connection_string)

# Define your SQL query
sql_query = """
    SELECT
        order_id,
        category_name,
        brand_name,
        product_name,
        quantity,
        i.list_price
    FROM
        sales.order_items i
        JOIN production.products p ON p.product_id = i.product_id
        JOIN production.categories c ON c.category_id = p.category_id
        JOIN production.brands b ON b.brand_id = p.brand_id
"""

# Fetch data from the database and load it into a DataFrame
df = pd.read_sql_query(sql_query, engine)

# Specify the folder to save Excel files
output_folder = r'C:\Users\danis\Desktop\Excel Files'

# Create an audit DataFrame to track the imports
audit_data = []

# Create Excel files with multiple sheets
category_groups = df.groupby('category_name')

for category_name, group_df in category_groups:
    excel_file = os.path.join(output_folder, f'{category_name}_data.xlsx')
    writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
    
    brand_groups = group_df.groupby('brand_name')
    for brand_name, brand_group_df in brand_groups:
        sheet_name = brand_name
        rows_imported = len(brand_group_df)
        
        # Append audit data
        audit_data.append({
            'ID': len(audit_data) + 1,
            'file_name': excel_file,
            'sheet_name': sheet_name,
            'rows_imported': rows_imported
        })
        
        # Write data to the sheet
        brand_group_df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    writer.save()

# Create the audit DataFrame
audit_df = pd.DataFrame(audit_data)

# Print the audit DataFrame
audit_df